In [1]:
import matplotlib as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
data_folder = 'data/'
dataset= pd.read_csv(data_folder+'data.csv')

# I. Helping Steps to get through the implementation
### - Reading the dataset
### - Visualizing Uncleaned Data from the dataset.
### - Acknowledging the Labels
### - Acknowledging the Features

### - Cleaning the data
### - Building the Pipeline to clean the data from the dataset.
### - Visualizing cleaned data from the dataset.

## Features/Labels
### - labels | 1/0
### - 5 numerical features
### - 8 categorical features

In [3]:
dataset.head(50)

,age,workSector,education,educationNum,statusMarriage,career,relationship,race,sex,gainedCapital,lostCapital,hoursPerWeek,country,income
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


# II. Data Preprocessing

In [4]:
# Creating the DataFrame for the preprocessed cleaned Dataset
cleaned_dataset = pd.DataFrame(columns=dataset.columns)

In [5]:
# Categorizing Features 
numerical_features = ['age','educationNum','gainedCapital','lostCapital','hoursPerWeek']
categorical_features = ['workSector','education','statusMarriage','career','relationship','race','sex','country']
label = ['income']
label_encoders = {}

In [6]:
def updating_series(dataset,feature,updated_feature):
    dataset[feature] = updated_feature

# Cleaning Categorical Features
[updating_series(dataset, feature, dataset[feature].apply(str.strip)) for feature in categorical_features]
dataset.head()

,age,workSector,education,educationNum,statusMarriage,career,relationship,race,sex,gainedCapital,lostCapital,hoursPerWeek,country,income
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [7]:
# Repopulating cleaned dataset
for feature in dataset.columns:
    if feature in numerical_features:
        cleaned_dataset[feature] = dataset[feature].astype(float)
        
# Transforming categorical features to one hot encoded
for feature in dataset.columns:
    if feature in categorical_features+label:
        label_encoders[feature] = LabelEncoder().fit(dataset[feature].astype(str))

# Repopulating cleaned dataset with the one hot encoded features
for feature in label_encoders.keys():
    cleaned_dataset[feature] = label_encoders[feature].transform(dataset[feature].astype(str))

In [8]:
# Visualizing Dataset
cleaned_dataset.head(50)

,age,workSector,education,educationNum,statusMarriage,career,relationship,race,sex,gainedCapital,lostCapital,hoursPerWeek,country,income
0,39.0,5,9,13.0,4,0,1,4,1,2174.0,0.0,40.0,38,0
1,50.0,4,9,13.0,2,3,0,4,1,0.0,0.0,13.0,38,0
2,38.0,2,11,9.0,0,5,1,4,1,0.0,0.0,40.0,38,0
3,53.0,2,1,7.0,2,5,0,2,1,0.0,0.0,40.0,38,0
4,28.0,2,9,13.0,2,9,5,2,0,0.0,0.0,40.0,4,0
5,37.0,2,12,14.0,2,3,5,4,0,0.0,0.0,40.0,38,0
6,49.0,2,6,5.0,3,7,1,2,0,0.0,0.0,16.0,22,0
7,52.0,4,11,9.0,2,3,0,4,1,0.0,0.0,45.0,38,1
8,31.0,2,12,14.0,4,9,1,4,0,14084.0,0.0,50.0,38,1
9,42.0,2,9,13.0,2,3,0,4,1,5178.0,0.0,40.0,38,1


In [9]:
# Removing outliers to enhance the dataset's features correlation importance during the training 
# part of the model's creation pipeline
    
print('Removing Outliers')

clf = IsolationForest(max_samples = 100, random_state = 0)
clf.fit(cleaned_dataset)
y_noano = clf.predict(cleaned_dataset)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

cleaned_dataset = cleaned_dataset.iloc[y_noano[y_noano['Top'] == 1].index.values]
cleaned_dataset.reset_index(drop = True, inplace = True)
print("Number of Outliers: {}".format(y_noano[y_noano['Top'] == -1].shape[0]))
print("Number of rows without outliers: {}".format(cleaned_dataset.shape[0]))

Removing Outliers
Number of Outliers: 8474
Number of rows without outliers: 36748


In [10]:
# Creating Features and Label Datasets
Y = cleaned_dataset.pop(label[0])
X = cleaned_dataset

In [11]:
# Creating Train, Val, Test Datasets
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X, Y, test_size=0.30)

X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.30)

In [12]:
# Saving data due to outliers removal from the datasets

print('Saving Train/Val/Test Datasets')

X_train.to_csv(data_folder+'X_train.csv',index=False,header=1)
X_val.to_csv(data_folder+'X_val.csv',index=False,header=1)
X_test.to_csv(data_folder+'X_test.csv',index=False,header=1)

Y_train.to_csv(data_folder+'Y_train.csv',index=False,header=1)
Y_val.to_csv(data_folder+'Y_val.csv',index=False,header=1)
Y_test.to_csv(data_folder+'Y_test.csv',index=False,header=1)

Saving Train/Val/Test Datasets


# III. Implementing the classification model 
## Tensorflow implementation due to feature layer transformations 

In [19]:
#Creating Feature Columns with Tensorflow Objects
def create_feature_columns(encoders):

    numeric_columns = {feature: tf.feature_column.numeric_column(feature) for feature in numerical_features}

    categorical_columns_with_vocabulary_list = {item: tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(key=item, vocabulary_list=encoders[item].classes_))
        for item in categorical_features}

    feature_columns = {}
    feature_columns.update(numeric_columns)
    feature_columns.update(categorical_columns_with_vocabulary_list)


    return feature_columns


# Creating the input function for the training of the model with Tensorflow
def build_input_fn_train():

    data_train = pd.read_csv(data_folder+'X_train.csv')
    data_train_label = pd.read_csv(data_folder+'Y_train.csv')
    
    [updating_series(data_train,feature,data_train[feature].astype(str)) for feature in categorical_features]
    
    return tf.estimator.inputs.pandas_input_fn(
        x= data_train,
        y= data_train_label.income,
        batch_size=64,
        num_epochs=80,
        shuffle=True,
        num_threads=8) 


# Creating the input function for the evaluation of the model with Tensorflow
def build_input_fn_val():
    
    data_val = pd.read_csv(data_folder+'X_val.csv')
    data_val_label = pd.read_csv(data_folder+'Y_val.csv')

    [updating_series(data_val,feature,data_val[feature].astype(str)) for feature in categorical_features]

    return tf.estimator.inputs.pandas_input_fn(
        x= data_val,
        y= data_val_label.income,
        shuffle=False)


# Creating the input function for the predictions of the model with Tensorflow
def build_input_fn_test():
    
    data_test = pd.read_csv(data_folder+'X_test.csv')
    data_test_label = pd.read_csv(data_folder+'Y_test.csv')

    [updating_series(data_test,feature,data_test[feature].astype(str)) for feature in categorical_features]

    return tf.estimator.inputs.pandas_input_fn(
        x= data_test,
        shuffle=False)

#Creating input receiver function for model export
def create_input_receiver_serving():
    def input_receiver():

        inputs = {}

        for column_name in numerical_features+categorical_features:
            if column_name in categorical_features:
                inputs[column_name] = tf.placeholder(shape=[None], dtype=tf.string)
            else:
                inputs[column_name] = tf.placeholder(shape=[None], dtype=tf.float32)

        features_dict = {key: tf.expand_dims(tensor, -1)for key, tensor in inputs.items()}

        from pprint import pprint
        pprint("****************************************")
        pprint("serving_feature_columns")
        pprint(features_dict)
        pprint("****************************************")

        return tf.estimator.export.ServingInputReceiver(
            features=features_dict,
            receiver_tensors= inputs
        )
        
    return input_receiver

In [14]:
# Creating Optimizer
def create_optimizer():
    return lambda: tf.train.AdamOptimizer(tf.train.exponential_decay(learning_rate=0.0001,global_step=tf.train.get_global_step(),decay_steps=8888,decay_rate=0.98))

In [15]:
# Creating classifier model
def create_regressor(encoders):

    estimator = tf.estimator.DNNLinearCombinedClassifier(
        model_dir='/classifier_model/',
        n_classes=2,
        dnn_feature_columns=create_feature_columns(label_encoders).values(),
        dnn_hidden_units=[512,512],
        dnn_optimizer=create_optimizer(),
        dnn_activation_fn=tf.nn.relu,
    )
    
    return estimator

In [16]:
estimator = create_regressor(label_encoders)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/classifier_model/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023C18E87F98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
estimator.train(input_fn=build_input_fn_train())

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /classifier_mo

INFO:tensorflow:global_step/sec: 148.588
INFO:tensorflow:loss = 26.002178, step = 244654 (0.673 sec)
INFO:tensorflow:global_step/sec: 146.628
INFO:tensorflow:loss = 34.414814, step = 244754 (0.682 sec)
INFO:tensorflow:global_step/sec: 145.349
INFO:tensorflow:loss = 17.21315, step = 244854 (0.687 sec)
INFO:tensorflow:global_step/sec: 147.493
INFO:tensorflow:loss = 25.188805, step = 244954 (0.678 sec)
INFO:tensorflow:global_step/sec: 146.843
INFO:tensorflow:loss = 25.413982, step = 245054 (0.682 sec)
INFO:tensorflow:global_step/sec: 146.412
INFO:tensorflow:loss = 21.53098, step = 245154 (0.683 sec)
INFO:tensorflow:global_step/sec: 146.2
INFO:tensorflow:loss = 20.012451, step = 245254 (0.683 sec)
INFO:tensorflow:global_step/sec: 146.413
INFO:tensorflow:loss = 23.237635, step = 245354 (0.683 sec)
INFO:tensorflow:global_step/sec: 148.588
INFO:tensorflow:loss = 24.367134, step = 245454 (0.674 sec)
INFO:tensorflow:global_step/sec: 149.254
INFO:tensorflow:loss = 21.118423, step = 245554 (0.670

INFO:tensorflow:loss = 19.649101, step = 252754 (0.694 sec)
INFO:tensorflow:global_step/sec: 139.276
INFO:tensorflow:loss = 30.154276, step = 252854 (0.718 sec)
INFO:tensorflow:global_step/sec: 142.653
INFO:tensorflow:loss = 24.977638, step = 252954 (0.701 sec)
INFO:tensorflow:global_step/sec: 146.843
INFO:tensorflow:loss = 20.663765, step = 253054 (0.681 sec)
INFO:tensorflow:global_step/sec: 141.243
INFO:tensorflow:loss = 20.334026, step = 253154 (0.708 sec)
INFO:tensorflow:global_step/sec: 133.333
INFO:tensorflow:loss = 23.238495, step = 253254 (0.749 sec)
INFO:tensorflow:global_step/sec: 129.033
INFO:tensorflow:loss = 28.164537, step = 253354 (0.776 sec)
INFO:tensorflow:global_step/sec: 126.904
INFO:tensorflow:loss = 30.902367, step = 253454 (0.788 sec)
INFO:tensorflow:global_step/sec: 126.262
INFO:tensorflow:loss = 24.834618, step = 253554 (0.792 sec)
INFO:tensorflow:global_step/sec: 132.803
INFO:tensorflow:loss = 29.132807, step = 253654 (0.753 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:global_step/sec: 126.586
INFO:tensorflow:loss = 23.062454, step = 260954 (0.790 sec)
INFO:tensorflow:global_step/sec: 132.979
INFO:tensorflow:loss = 16.608606, step = 261054 (0.752 sec)
INFO:tensorflow:global_step/sec: 130.378
INFO:tensorflow:loss = 18.422531, step = 261154 (0.767 sec)
INFO:tensorflow:global_step/sec: 131.926
INFO:tensorflow:loss = 20.03186, step = 261254 (0.758 sec)
INFO:tensorflow:global_step/sec: 130.378
INFO:tensorflow:loss = 28.134285, step = 261354 (0.767 sec)
INFO:tensorflow:global_step/sec: 131.061
INFO:tensorflow:loss = 32.07427, step = 261454 (0.767 sec)
INFO:tensorflow:global_step/sec: 128.7
INFO:tensorflow:loss = 22.458015, step = 261554 (0.774 sec)
INFO:tensorflow:global_step/sec: 130.21
INFO:tensorflow:loss = 15.445292, step = 261654 (0.770 sec)
INFO:tensorflow:global_step/sec: 125.47
INFO:tensorflow:loss = 26.281437, step = 261754 (0.793 sec)
INFO:tensorflow:global_step/sec: 132.275
INFO:tensorflow:loss = 22.450203, step = 261854 (0.757 s

INFO:tensorflow:global_step/sec: 142.45
INFO:tensorflow:loss = 23.217476, step = 269154 (0.702 sec)
INFO:tensorflow:global_step/sec: 147.059
INFO:tensorflow:loss = 17.647163, step = 269254 (0.680 sec)
INFO:tensorflow:global_step/sec: 145.138
INFO:tensorflow:loss = 24.513603, step = 269354 (0.690 sec)
INFO:tensorflow:global_step/sec: 145.561
INFO:tensorflow:loss = 20.985142, step = 269454 (0.686 sec)
INFO:tensorflow:global_step/sec: 147.059
INFO:tensorflow:loss = 22.868, step = 269554 (0.680 sec)
INFO:tensorflow:global_step/sec: 143.261
INFO:tensorflow:loss = 26.289263, step = 269654 (0.698 sec)
INFO:tensorflow:global_step/sec: 136.799
INFO:tensorflow:loss = 34.771935, step = 269754 (0.730 sec)
INFO:tensorflow:global_step/sec: 148.589
INFO:tensorflow:loss = 21.912716, step = 269854 (0.674 sec)
INFO:tensorflow:global_step/sec: 148.809
INFO:tensorflow:loss = 19.24033, step = 269954 (0.672 sec)
INFO:tensorflow:global_step/sec: 146.843
INFO:tensorflow:loss = 21.364162, step = 270054 (0.681 

...

INFO:tensorflow:loss = 23.659576, step = 529554 (0.677 sec)
INFO:tensorflow:global_step/sec: 146.844
INFO:tensorflow:loss = 22.913836, step = 529654 (0.681 sec)
INFO:tensorflow:global_step/sec: 147.929
INFO:tensorflow:loss = 22.214321, step = 529754 (0.676 sec)
INFO:tensorflow:global_step/sec: 147.929
INFO:tensorflow:loss = 23.507778, step = 529854 (0.676 sec)
INFO:tensorflow:global_step/sec: 147.275
INFO:tensorflow:loss = 27.648518, step = 529954 (0.679 sec)
INFO:tensorflow:global_step/sec: 144.925
INFO:tensorflow:loss = 21.725979, step = 530054 (0.690 sec)
INFO:tensorflow:global_step/sec: 149.254
INFO:tensorflow:loss = 24.710236, step = 530154 (0.670 sec)
INFO:tensorflow:global_step/sec: 148.371
INFO:tensorflow:loss = 20.230446, step = 530254 (0.673 sec)
INFO:tensorflow:global_step/sec: 148.588
INFO:tensorflow:loss = 25.539038, step = 530354 (0.674 sec)
INFO:tensorflow:global_step/sec: 148.146
INFO:tensorflow:loss = 13.285908, step = 530454 (0.675 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:global_step/sec: 145.773
INFO:tensorflow:loss = 14.877655, step = 537754 (0.686 sec)
INFO:tensorflow:global_step/sec: 147.929
INFO:tensorflow:loss = 40.173843, step = 537854 (0.675 sec)
INFO:tensorflow:global_step/sec: 147.058
INFO:tensorflow:loss = 23.799189, step = 537954 (0.680 sec)
INFO:tensorflow:global_step/sec: 148.368
INFO:tensorflow:loss = 22.280975, step = 538054 (0.675 sec)
INFO:tensorflow:global_step/sec: 147.71
INFO:tensorflow:loss = 32.014465, step = 538154 (0.677 sec)
INFO:tensorflow:global_step/sec: 145.349
INFO:tensorflow:loss = 20.838146, step = 538254 (0.688 sec)
INFO:tensorflow:global_step/sec: 148.588
INFO:tensorflow:loss = 16.592693, step = 538354 (0.673 sec)
INFO:tensorflow:global_step/sec: 147.71
INFO:tensorflow:loss = 21.605179, step = 538454 (0.676 sec)
INFO:tensorflow:global_step/sec: 148.148
INFO:tensorflow:loss = 23.633507, step = 538554 (0.676 sec)
INFO:tensorflow:global_step/sec: 148.588
INFO:tensorflow:loss = 22.508026, step = 538654 (0.6

INFO:tensorflow:loss = 17.926643, step = 545854 (0.710 sec)
INFO:tensorflow:global_step/sec: 122.85
INFO:tensorflow:loss = 20.603762, step = 545954 (0.815 sec)
INFO:tensorflow:global_step/sec: 127.388
INFO:tensorflow:loss = 20.788162, step = 546054 (0.785 sec)
INFO:tensorflow:global_step/sec: 126.583
INFO:tensorflow:loss = 23.901962, step = 546154 (0.789 sec)
INFO:tensorflow:global_step/sec: 133.511
INFO:tensorflow:loss = 26.389442, step = 546254 (0.749 sec)
INFO:tensorflow:global_step/sec: 136.054
INFO:tensorflow:loss = 28.345057, step = 546354 (0.735 sec)
INFO:tensorflow:global_step/sec: 140.057
INFO:tensorflow:loss = 24.162876, step = 546454 (0.714 sec)
INFO:tensorflow:global_step/sec: 141.243
INFO:tensorflow:loss = 23.280178, step = 546554 (0.707 sec)
INFO:tensorflow:global_step/sec: 141.044
INFO:tensorflow:loss = 16.563597, step = 546654 (0.709 sec)
INFO:tensorflow:global_step/sec: 144.718
INFO:tensorflow:loss = 24.452965, step = 546754 (0.692 sec)
INFO:tensorflow:global_step/sec:

INFO:tensorflow:global_step/sec: 146.629
INFO:tensorflow:loss = 31.112743, step = 554054 (0.682 sec)
INFO:tensorflow:global_step/sec: 146.843
INFO:tensorflow:loss = 24.964828, step = 554154 (0.681 sec)
INFO:tensorflow:global_step/sec: 148.147
INFO:tensorflow:loss = 21.177485, step = 554254 (0.675 sec)
INFO:tensorflow:global_step/sec: 146.408
INFO:tensorflow:loss = 18.379942, step = 554354 (0.683 sec)
INFO:tensorflow:global_step/sec: 147.934
INFO:tensorflow:loss = 14.555943, step = 554454 (0.676 sec)
INFO:tensorflow:global_step/sec: 148.148
INFO:tensorflow:loss = 25.106262, step = 554554 (0.675 sec)
INFO:tensorflow:global_step/sec: 146.627
INFO:tensorflow:loss = 24.97472, step = 554654 (0.682 sec)
INFO:tensorflow:global_step/sec: 147.493
INFO:tensorflow:loss = 21.76651, step = 554754 (0.678 sec)
INFO:tensorflow:global_step/sec: 148.589
INFO:tensorflow:loss = 23.834717, step = 554854 (0.673 sec)
INFO:tensorflow:global_step/sec: 145.349
INFO:tensorflow:loss = 23.655422, step = 554954 (0.6

INFO:tensorflow:loss = 22.555641, step = 562154 (0.678 sec)
INFO:tensorflow:global_step/sec: 147.275
INFO:tensorflow:loss = 22.101034, step = 562254 (0.679 sec)
INFO:tensorflow:global_step/sec: 146.412
INFO:tensorflow:loss = 17.199934, step = 562354 (0.684 sec)
INFO:tensorflow:global_step/sec: 145.35
INFO:tensorflow:loss = 22.153217, step = 562454 (0.688 sec)
INFO:tensorflow:global_step/sec: 147.058
INFO:tensorflow:loss = 21.030413, step = 562554 (0.679 sec)
INFO:tensorflow:global_step/sec: 147.929
INFO:tensorflow:loss = 24.773798, step = 562654 (0.676 sec)
INFO:tensorflow:Saving checkpoints for 562692 into /classifier_model/model.ckpt.
INFO:tensorflow:Loss for final step: 8.058189.


In [20]:
estimator.evaluate(input_fn=build_input_fn_val())

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-01T18:06:09Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /classifier_model/model.ckpt-562692
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-07-01-18:06:10
INFO:tensorflow:Saving dict for global step 562692: accuracy = 0.82817155, accuracy_baseline = 0.77957755, auc = 0.8398658, auc_precision_recall = 0.6438353, average_loss = 0.3889269, global_step = 562692, label/mean = 0.22042245, loss = 49.202442, precision = 0.69799364, prediction/mean = 0.21214576, recall = 0.38859496
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 562692: /classifier_model/model.ckpt-562692


{'accuracy': 0.82817155,
 'accuracy_baseline': 0.77957755,
 'auc': 0.8398658,
 'auc_precision_recall': 0.6438353,
 'average_loss': 0.3889269,
 'label/mean': 0.22042245,
 'loss': 49.202442,
 'precision': 0.69799364,
 'prediction/mean': 0.21214576,
 'recall': 0.38859496,
 'global_step': 562692}

In [21]:
# Verifying accuracy of predictions on the test dataset
predictions = estimator.predict(input_fn=build_input_fn_test())
preds = [item['classes'] for item in predictions]

results = pd.read_csv(data_folder+'Y_test.csv')
results['predictions'] = [int(item[0]) for item in preds]
results['valid_predictions'] = results.apply(lambda x: 1 if x.income == x.predictions else 0,axis=1)
results_count = results.valid_predictions.value_counts()
percentage = (results_count[1] * 100)/(results_count[0] + results_count[1] )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /classifier_model/model.ckpt-562692
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [22]:
percentage

83.31318016928658

In [23]:
estimator.export_savedmodel('saved_model', serving_input_receiver_fn=create_input_receiver_serving())

Instructions for updating:
This function has been renamed, use `export_saved_model` instead.
'****************************************'
'serving_feature_columns'
{'age': <tf.Tensor 'ExpandDims:0' shape=(?, 1) dtype=float32>,
 'career': <tf.Tensor 'ExpandDims_8:0' shape=(?, 1) dtype=string>,
 'country': <tf.Tensor 'ExpandDims_12:0' shape=(?, 1) dtype=string>,
 'education': <tf.Tensor 'ExpandDims_6:0' shape=(?, 1) dtype=string>,
 'educationNum': <tf.Tensor 'ExpandDims_1:0' shape=(?, 1) dtype=float32>,
 'gainedCapital': <tf.Tensor 'ExpandDims_2:0' shape=(?, 1) dtype=float32>,
 'hoursPerWeek': <tf.Tensor 'ExpandDims_4:0' shape=(?, 1) dtype=float32>,
 'lostCapital': <tf.Tensor 'ExpandDims_3:0' shape=(?, 1) dtype=float32>,
 'race': <tf.Tensor 'ExpandDims_10:0' shape=(?, 1) dtype=string>,
 'relationship': <tf.Tensor 'ExpandDims_9:0' shape=(?, 1) dtype=string>,
 'sex': <tf.Tensor 'ExpandDims_11:0' shape=(?, 1) dtype=string>,
 'statusMarriage': <tf.Tensor 'ExpandDims_7:0' shape=(?, 1) dtype=str

b'saved_model\\1593615983'

In [142]:
import json
def financialstatus_data(person_data):
    person_data = person_data.to_dict()
    [person_data.update({feature: str(int(person_data[feature]))}) for feature in categorical_features]
    
    output = json.dumps({"signature_name": "predict","instances":[person_data]})
    
    return output


def check_financialstatus(person_data):
    person_data = str(person_data).replace("'",'"')
    r = requests.post("http://localhost:8501/v1/models/financialstatus/versions/1:predict", data=person_data)
    
    result = r.json()
    
    return result['predictions'][0]['class_ids'][0]

In [143]:
t_data = pd.read_csv(data_folder+'X_test.csv')

t_data['financialstatus'] = [check_financialstatus(financialstatus_preprocessing(t_data.iloc[index])) for index in range(len(t_data))]
t_data['financialstatus'] = label_encoders['income'].inverse_transform(t_data['financialstatus'].values)
t_data['financialstatus'] 

0       <=50K
1       <=50K
2        >50K
3       <=50K
4       <=50K
        ...  
3303    <=50K
3304    <=50K
3305     >50K
3306    <=50K
3307     >50K
Name: financialstatus, Length: 3308, dtype: object